<a href="https://colab.research.google.com/github/Letch49/ML-vvsu-2025/blob/master/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_6_(%D0%9A%D0%BB%D0%B0%D1%81%D1%82%D0%B5%D1%80%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# МЛ практика 6. Обучение без учителя

## **Цель занятия:**
Изучить методы обучения без учителя, познакомиться с алгоритмами кластеризации, понять их применение и реализовать модели кластеризации с использованием различных алгоритмов.

## **Задание состоит из следующих частей:**
1. Определение обучения без учителя
2. Разбор алгоритмов DBSSCAN, K-means
3. Метрики качества кластеризации
4. Пример полезных задач, которые решает кластеризация

**P.S.** Некоторые вещи (например PCA, обработка текстовых данных) мы пройдем чуть позже.

## Задачи для домашки ⏰

1. Преобразовать датасет
2. Построить кластеры
3. Описать результат полученных кластеров (для K-Means и DBSSCAN)
4. Объяснить метрики качества

**По желанию**

1. Спарсить данные по статьям (например lenta.ru или naked-science.ru)
2. Преобразовать текста статьи (по принципу описанному в последнем пункте)
3. Оценить метрики качества, провести аналитику кластеров (можно не максимально подробно, если получилось слишком много кластеров)

## Предобработка (все по классике)

Открываем датасет, преобразуем данные, идем дальше

<pre>AGE_50 – Возраст человека.
MD_50 – Медицинский диагноз или индекс здоровья.
SBP_50 – Систолическое артериальное давление (systolic blood pressure) .
DBP_50 – Диастолическое артериальное давление (diastolic blood pressure).
HT_50 – Рост (height).
WT_50 – Вес (weight).
CHOL_50 – Уровень холестерина (cholesterol).
SES – Социально-экономический статус (socioeconomic status).
CL_STATUS – Клинический статус (возможно, индикатор состояния здоровья или наличия хронических заболеваний).
MD_62 – Медицинский диагноз или индекс здоровья.
SBP_62 – Систолическое артериальное давление.
DBP_62 – Диастолическое артериальное давление.
CHOL_62 – Уровень холестерина.
WT_62 – Вес.
IHD_DX – Диагноз ишемической болезни сердца (ischemic heart disease diagnosis).
DEATH – Индикатор смерти (возможно, "1" означает смерть, "0" — нет).</pre>

- **Непрерывные переменные:**
     - `AGE_50, SBP_50', DBP_50, HT_50, WT_50, SBP_62, DBP_62, CHOL_50, CHOL_62, WT_62`
   
- **Дискретные переменные:**
     - `MD_50, SES, CL_STATUS, MD_62, IHD_DX, DEATH`

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Загрузка набора данных
df = pd.read_csv('heart.csv')

In [41]:
df[['MD_50', 'SES', 'CL_STATUS', 'MD_62', 'IHD_DX', 'DEATH']] = df[['MD_50', 'SES', 'CL_STATUS', 'MD_62', 'IHD_DX', 'DEATH']].astype({'MD_50':'category', 'SES':'category', 'CL_STATUS':'category', 'MD_62':'category', 'IHD_DX':'category', 'DEATH':'category'})
df[['SBP_50', 'DBP_50', 'WT_50', 'HT_50', 'CHOL_50', 'SBP_62', 'DBP_62', 'CHOL_62', 'WT_62']] = df[['SBP_50', 'DBP_50', 'WT_50', 'HT_50', 'CHOL_50', 'SBP_62', 'DBP_62', 'CHOL_62', 'WT_62']].astype({'SBP_50':'float64', 'DBP_50':'float64','WT_50':'float64', 'HT_50':'float64', 'CHOL_50':'float64', 'SBP_62':'float64', 'DBP_62':'float64', 'CHOL_62':'float64', 'WT_62':'float64'})

In [42]:
print(df.dtypes)

AGE_50          int64
MD_50        category
SBP_50        float64
DBP_50        float64
HT_50         float64
WT_50         float64
CHOL_50       float64
SES          category
CL_STATUS    category
MD_62        category
SBP_62        float64
DBP_62        float64
CHOL_62       float64
WT_62         float64
IHD_DX       category
DEATH        category
dtype: object


### Преобразование категориальных фич

In [43]:
# Создаем копию данных
category_columns: list[str] = df.select_dtypes(include=['category']).columns # собираем колонки помеченные как objects

# Применяем One-Hot Encoding
df = pd.get_dummies(df, columns=category_columns, drop_first=True)
print(df.dtypes)

AGE_50           int64
SBP_50         float64
DBP_50         float64
HT_50          float64
WT_50          float64
CHOL_50        float64
SBP_62         float64
DBP_62         float64
CHOL_62        float64
WT_62          float64
MD_50_2          uint8
MD_50_3          uint8
MD_50_4          uint8
SES_2            uint8
SES_3            uint8
SES_4            uint8
SES_5            uint8
CL_STATUS_3      uint8
CL_STATUS_4      uint8
CL_STATUS_5      uint8
CL_STATUS_6      uint8
CL_STATUS_7      uint8
CL_STATUS_8      uint8
MD_62_2          uint8
MD_62_3          uint8
MD_62_4          uint8
MD_62_5          uint8
IHD_DX_1         uint8
IHD_DX_2         uint8
IHD_DX_3         uint8
IHD_DX_4         uint8
IHD_DX_5         uint8
IHD_DX_6         uint8
IHD_DX_7         uint8
IHD_DX_8         uint8
IHD_DX_9         uint8
DEATH_1          uint8
dtype: object


In [44]:
df.type_white = df.DEATH_1.astype(int)

C:\Users\Thunderobot\AppData\Local\Temp\ipykernel_3652\26842141.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.type_white = df.DEATH_1.astype(int)


### Преобразование непрерывных фич

In [45]:
from sklearn.preprocessing import MinMaxScaler
# Выбираем числовые признаки
numeric_features = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Инициализируем scaler
scaler = MinMaxScaler()

# Применяем нормализацию
df[numeric_features] = scaler.fit_transform(df[numeric_features])


In [46]:
df

,AGE_50,SBP_50,DBP_50,HT_50,WT_50,CHOL_50,SBP_62,DBP_62,CHOL_62,WT_62,...,IHD_DX_1,IHD_DX_2,IHD_DX_3,IHD_DX_4,IHD_DX_5,IHD_DX_6,IHD_DX_7,IHD_DX_8,IHD_DX_9,DEATH_1
0,0.448980,0.180328,0.264706,0.214286,0.279412,0.412048,0.31250,0.318841,0.576419,0.275362,...,0,1,0,0,0,0,0,0,0,1
1,0.673469,0.344262,0.333333,0.571429,0.426471,0.380723,0.32500,0.173913,0.484716,0.413043,...,0,0,0,0,0,0,0,0,1,1
2,0.673469,0.262295,0.509804,0.642857,0.830882,0.534940,0.38750,0.492754,0.720524,0.833333,...,0,1,0,0,0,0,0,0,0,1
3,0.571429,0.262295,0.411765,0.785714,0.882353,0.286747,0.30000,0.173913,0.305677,0.862319,...,0,0,1,0,0,0,0,0,0,1
4,0.673469,0.245902,0.352941,0.357143,0.183824,0.296386,0.30000,0.000000,0.532751,0.217391,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.612245,0.221311,0.411765,0.357143,0.286765,0.433735,0.28125,0.130435,0.585153,0.318841,...,0,0,0,0,0,0,0,0,0,0
196,0.061224,0.180328,0.313725,0.571429,0.205882,0.000000,0.26250,0.289855,0.257642,0.326087,...,0,0,0,0,0,0,0,0,0,0
197,0.000000,0.344262,0.411765,0.357143,0.301471,0.216867,0.37500,0.420290,0.589520,0.362319,...,0,0,0,0,0,0,0,0,0,0
198,0.530612,0.426230,0.450980,0.357143,0.213235,0.024096,0.48750,0.463768,0.091703,0.355072,...,0,0,0,0,0,0,0,0,0,0


In [47]:
df[['MD_50_2', 'MD_50_3', 'MD_50_3']]

,MD_50_2,MD_50_3,MD_50_3
0,0,0,0
1,0,0,0
2,1,0,0
3,0,0,0
4,0,1,1
...,...,...,...
195,0,0,0
196,0,0,0
197,0,1,1
198,0,1,1


## О кластеризации и задачи обучения без учителя

#### Обучение с учителем

Представьте, что у вас есть список задач с правильными ответами. Например, вам дали примеры: «Это яблоко», «Это груша».

Задача — научиться отличать их, чтобы потом, когда вы увидите новый фрукт, вы смогли правильно его назвать. То есть у вас есть **учитель** — тот, кто заранее всё объяснил и дал правильные ответы.  

#### Обучение без учителя

А теперь представьте, что вам дали корзину фруктов, но никто не сказал, как они называются.

Задача — посмотреть на фрукты и найти, что у них общего, например: круглые положить в одну группу, длинные в другую. Тут вы работаете **без подсказок** и сами решаете, как группировать.  

#### Разница

- В задачах с учителем есть правильные ответы, и мы обучем алгоритм делать предсказания на основе примеров.  
- В задачах без учителя правильных ответов нет, ищем закономерности и пытаемся найти смысл группировки данные самостоятельно.

Пример:


1. **С учителем**: Мы ищем лучшее вино.  
2. **Без учителя**: Мы пытаемся объеденить вина, чтобы понять по какому принципу они объеденяются.  

## K-means

**K-means** — это алгоритм кластеризации, который делит данные на 𝑘 групп (кластеров). Основная идея: найти центры кластеров так, чтобы точки внутри одного кластера были ближе друг к другу, чем к точкам из других кластеров.

1. Инициализация: выбираем 𝑘 случайных центров (центроидов).
2. Присвоение точек: каждая точка относится к ближайшему центру.
Обновление центров: пересчитываем центры кластеров как среднее всех точек в кластере.
3. Повторяем шаги 2–3, пока центры не перестанут меняться или не достигнем максимального числа итераций.

In [48]:
! pip install ipywidgets


In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

def interactive_kmeans_visualization(X, max_clusters=10, max_iterations=50):

    # преобразуем X в numpy array
    if isinstance(X, pd.DataFrame):
        X = X.values

    def plot_kmeans_iteration(X, centroids, labels=None, step=0):
        clear_output(wait=True)
        plt.figure(figsize=(8, 6))
        if labels is not None:
            plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', s=30, alpha=0.6)
        else:
            plt.scatter(X[:, 0], X[:, 1], s=30, alpha=0.6)
        plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x', s=100, label='Centroids')
        plt.title(f'K-means Iteration {step}')
        plt.legend()
        plt.show()

    def k_means_interactive(X, k, max_iters):
        np.random.seed(42)
        centroids = X[np.random.choice(X.shape[0], k, replace=False)]
        for step in range(max_iters):
            distances = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
            labels = np.argmin(distances, axis=1)
            plot_kmeans_iteration(X, centroids, labels, step)
            new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(k)])
            if np.all(centroids == new_centroids):
                break
            centroids = new_centroids

    def run_kmeans(k, max_iters):
        k_means_interactive(X, k, max_iters)

    k_slider = widgets.IntSlider(value=3, min=2, max=max_clusters, step=1, description='Clusters (k):')
    iter_slider = widgets.IntSlider(value=10, min=1, max=max_iterations, step=1, description='Max Iterations:')

    interactive_plot = widgets.interactive(run_kmeans, k=k_slider, max_iters=iter_slider)
    display(interactive_plot)

# Берем датасет и визуализируем K-means
interactive_kmeans_visualization(df[:100])


interactive(children=(IntSlider(value=3, description='Clusters (k):', max=10, min=2), IntSlider(value=10, desc…

### K-means через реализацию skelarn

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Определяем количество кластеров k
k = 3  # Вы можете изменить это значение в соответствии с вашими данными

# Создаем и обучаем модель KMeans
kmeans = KMeans(n_clusters=k, random_state=42)
labels = kmeans.fit_predict(df)
centroids = kmeans.cluster_centers_

# Добавляем метки кластеров в исходный DataFrame
df_k_means = df.copy()
df_k_means['Cluster'] = labels

# Вывод статистики по кластерам для каждого признака
print("Статистика по кластерам:")
for column in df.columns:
    print(f"\nСтатистика для признака '{column}':")
    cluster_stats = df_k_means.groupby('Cluster')[column].agg(['mean', 'std', 'min', 'max'])
    print(cluster_stats)

# Вычисление коэффициента силуэта
if k > 1:
    silhouette_avg = silhouette_score(df, labels)
    print(f"\nКоэффициент силуэта для k={k}: {silhouette_avg:.4f}")
else:
    print("\nКоэффициент силуэта не определен для k=1")


D:\Program Files\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
D:\Program Files\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Статистика по кластерам:

Статистика для признака 'AGE_50':
             mean       std       min       max
Cluster                                        
0        0.564286  0.222066  0.040816  1.000000
1        0.508103  0.226931  0.000000  0.979592
2        0.527494  0.213209  0.061224  0.897959

Статистика для признака 'SBP_50':
             mean       std       min       max
Cluster                                        
0        0.327596  0.175884  0.000000  1.000000
1        0.307257  0.131167  0.057377  0.836066
2        0.272427  0.135600  0.098361  0.918033

Статистика для признака 'DBP_50':
             mean       std       min       max
Cluster                                        
0        0.462255  0.109184  0.215686  0.705882
1        0.447809  0.110237  0.000000  0.754902
2        0.427696  0.103264  0.264706  1.000000

Статистика для признака 'HT_50':
             mean       std       min       max
Cluster                                        
0        0.463095  0

#### **Анализ кластеров**



После применения алгоритма K-means с количеством кластеров \( k=3 \), мы получили три группы пациентов с заметными отличиями по некоторым признакам. Рассмотрим признаки, по которым кластеры существенно различаются:

1. **AGE_50 (Возраст):**
   - **Кластер 0** и **Кластер 2** Попали исключительно пожилые люди. при этом в кластере 2 люди старше, чем в кластере 0.
   - **Кластер 2** состоит из людей всех возрастов. 
   - **Вывод:** Можно предположить, что возраст не являлся решающим критерием для разделения.

2. **SBP_50 (Систолическое давление):**
   - **Кластер 0** имеет самый большой разброс (`std ≈ 0.175884`) и в среднем давление в этом кластере выше.
   - **Кластер 1** имеет более низкие минимальное и максимальное значения, чем в **Кластере 2**, хотя среднее значение в нём немного выше.
   - **Вывод:** В **Кластере 2** находятся люди с более высоким систолическим давлением. **Кластер 0** не имеет чётко выраженной закономерности.

3. **DBP_50 (Дисталическое давление):**
   - **Кластер 0** обладает самым высоким средним значением, однако минимальное и максимальное значение у него у него находятся ближе к средним, чем крайние значения в других кластерах. Минимальное и максимальное значение в **Кластере 1** ниже, чем в **Кластере 2**.
   - **Вывод:** Аналогичен предыдущему пункту.
   
Признаки 'HT_50' и 'WT_50' не имеют выраженной закономерности, поэтому пропустим их анализ.

4. **CHOL_50 (Уровень холестерина)**
    - В **Кластер 0** попали люди с высоким уровнем холестерина. Данный кластер имеет самые высокие среднее, минимальное и маскимальное значения, хотя и самый большой разброс.
    - В **Кластер 1** и **Кластер 2** попали люди со средним и нищким уровнем холестирина соответсвенно. Оба кластера имеют приблизителньо одинаковый разброс.
    - **Вывод:** Уровень холестирина являлся одним из наиболее значимых признаокв для кластеризации, т.к. наблюдается четкое разделение.
    
Признаки 'SBP_62', 'DBP_62', 'CHOL_62' и 'WT_62' по большей части повторяют признаки 'SBP_50', 'DBP_50', 'CHOL_50' и 'WT_50' соответвенно.

5. **CHOL_62 (Уровень холестерина)** 
    Сам по себе признак не имеет четкой закономерности между кластерами, однако её можно заметить, проведя сравнение с признаком CHOL_50.
    - В **Кластере 0** среднее значение меньше, чем в других кластерах, однако максимальное значение все еще самое большое. Минимальное значение меньше, чем в Кластере 1, на больше чем в Кластере 2. 
    - В **Кластерах 1 и 2** примерно одинаковые средние значения и разбросы, минимальные и максимальные значения разнятся между собой примерно на одинаковое значение (0,04).
    - **Вывод:** Можно предположить, что в **Кластерах 1 и 2** оказались люди, давление которых стабильно увеличилось с возрастом в отличие от **Кластера 0**.
    

**Коэффициент силуэта для \( k=3 \):  0.1111**

Коэффициент силуэта — это метрика для оценки качества кластеризации, показывающая, насколько объекты внутри одного кластера похожи друг на друга и отличаются от объектов в других кластерах.

- **Диапазон значений:** от -1 до 1.
  - **Близкие к 1:** Хорошая кластеризация; объекты внутри кластеров схожи, кластеры хорошо разделены.
  - **Около 0:** Кластеры пересекаются; объекты находятся на границах кластеров.
  - **Отрицательные значения:** Плохая кластеризация; объекты могут быть отнесены к неправильным кластерам.

**Интерпретация полученного значения:**

- **0.1111** — это положительное значение, указывающее на спорное качество кластеризации.
- Кластеры имеют достаточно много пересечений.

### DBSSCAN

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) — это алгоритм кластеризации, основанный на плотности точек в пространстве.

Параметры алгоритма:

- **Epsilon (eps)**: радиус окрестности точки.
- **Min_samples**: минимальное количество точек в eps-окрестности для того, чтобы точка считалась ядром кластера.

Классификация точек:

- **Core point (ядро)**: если в eps-окрестности точки находится не менее min_samples точек, то она является ядром.
- **Border point (граничная точка)**: точка, которая находится в eps-окрестности ядра, но сама не является ядром.
- **Noise point (шум)**: точка, которая не является ни ядром, ни граничной точкой. (**входит в кластер -1**)

Точки, которые связаны по плотности (через ядра и их eps-окрестности), объединяются в один кластер.

In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display, clear_output
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

def interactive_dbscan_visualization(X):
    if isinstance(X, pd.DataFrame):
        X = X.values

    def plot_dbscan(X, labels, core_samples_mask, eps, min_samples, metric):
        clear_output(wait=True)
        unique_labels = set(labels)
        colors = [plt.cm.Spectral(each)
                  for each in np.linspace(0, 1, len(unique_labels))]
        plt.figure(figsize=(8, 6))
        for k, col in zip(unique_labels, colors):
            if k == -1:
                # Черный цвет для шума
                col = [0, 0, 0, 0.2]

            class_member_mask = (labels == k)

            xy = X[class_member_mask & core_samples_mask]
            plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                     markeredgecolor='k', markersize=14)

            xy = X[class_member_mask & ~core_samples_mask]
            plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                     markeredgecolor='k', markersize=6)

        plt.title(f'DBSCAN: eps={eps}, min_samples={min_samples}, metric={metric}')
        plt.show()

    def run_dbscan(eps, min_samples, metric):
        # Обучение DBSCAN
        db = DBSCAN(eps=eps, min_samples=min_samples, metric=metric)
        db.fit(X)
        labels = db.labels_
        core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        core_samples_mask[db.core_sample_indices_] = True
        plot_dbscan(X, labels, core_samples_mask, eps, min_samples, metric)

    # Виджеты для настройки параметров
    eps_slider = widgets.FloatSlider(value=0.5, min=0.1, max=1.0, step=0.05, description='eps:')
    min_samples_slider = widgets.IntSlider(value=5, min=1, max=20, step=1, description='min_samples:')
    metric_dropdown = widgets.Dropdown(options=['euclidean', 'manhattan', 'chebyshev', 'minkowski'],
                                       value='euclidean', description='metric:')

    ui = widgets.VBox([eps_slider, min_samples_slider, metric_dropdown])
    out = widgets.interactive_output(run_dbscan, {'eps': eps_slider, 'min_samples': min_samples_slider, 'metric': metric_dropdown})
    display(ui, out)

# Пример использования
interactive_dbscan_visualization(df[:120])


Output()

### DBSSCAN через sklearn

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

# Параметры DBSCAN
eps = 1.5
min_samples = 4
metric = 'euclidean'

# Обучение модели DBSCAN
db = DBSCAN(eps=eps, min_samples=min_samples, metric=metric)
labels = db.fit_predict(df)

# Добавляем метки кластеров в DataFrame
df_dbscan = df.copy()
df_dbscan['Cluster'] = labels

# Количество кластеров (исключая шум)
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print(f"Количество кластеров: {n_clusters_}")

# Вывод статистики по кластерам
print("Статистика по кластерам:")
for column in df.columns:
    print(f"\nСтатистика для признака '{column}':")
    cluster_stats = df_dbscan[df_dbscan['Cluster'] != -1].groupby('Cluster')[column].agg(['mean', 'std', 'min', 'max', 'count'])
    print(cluster_stats)

# Вычисление коэффициента силуэта
if n_clusters_ > 1:
    silhouette_avg = silhouette_score(df[df_dbscan['Cluster'] != -1], labels[df_dbscan['Cluster'] != -1])
    print(f"\nКоэффициент силуэта для DBSCAN: {silhouette_avg:.4f}")
else:
    print("\nКоэффициент силуэта не определен, так как найден один кластер или данные являются шумом.")


# Визуализация результатов (с использованием PCA для снижения размерности)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(df)
df_pca = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
df_pca['Cluster'] = labels

plt.figure(figsize=(8, 6))
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

for k, col in zip(unique_labels, colors):
    class_member_mask = (labels == k)
    if k == -1:
        # Черный цвет для шума
        col = [0, 0, 0, 0.2]
    plt.plot(df_pca.loc[class_member_mask, 'PC1'],
             df_pca.loc[class_member_mask, 'PC2'],
             'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title(f'DBSCAN: eps={eps}, min_samples={min_samples}, metric={metric}')
plt.xlabel('Главная компонента 1')
plt.ylabel('Главная компонента 2')
plt.show()


NameError: name 'df' is not defined

#### **Анализ кластеров**

После применения алгоритма DBSCAN мы получили **два кластера**, между которыми наблюдаются существенные различия по некоторым признакам. Рассмотрим три наиболее значимых признака:

1. **AGE_50** 
    - **Кластер 0** состоит из людей среднего и пожилого возраста.
    - **Кластер 1** сосотоит преимущетвенно из людей среднего возраста, но включает в себя также детей.
    - **Кластер 2** состоит из молодых людей и людей среднего возраста.
2. **SBP_50** 
    - **Кластер 0** состоит из людей с низким и средним систолическим давлением.
    - **Кластер 1** состоит из людей с средним систолическим давлением.
    - **Кластер 2** состоит из людей с низким систолическим давлением. 
3. **DBP_50**
    - **Кластеры 0 и 2** имеют довольно похожие статистические показатели.
    - **Кластер 1** отличается от них более высокими средним и  минмальным значением.
4. **HT_50** 
    - **Кластер 0** состоит из людей среднего роста и высоких.
    - **Кластер 1** состоит из людей среднего роста.
    - **Кластер 2** состоит из людей низкого роста.
5. **WT_50**
    - **Кластер 0** состоит из с средним и большим весом.
    - **Кластеры 1 и 2** состоят из людей со средним весом.
6. **CHOL_50**
    По данному параметру кластеры существенно не различаются.    
7. **SBP_62**
    По данному параметру кластеры существенно не различаются.
8. **DBP_62** 
    - **Кластер 0** состоит из людей с средним систолическим давлением.
    - **Кластер 1** состоит из людей с низким и средним систолическим давлением.
    - **Кластер 2** состоит из людей с низким и  высоким систолическим давлением. 
9. **CHOL_62** 
    - **Кластер 0** состоит из людей с низким и средним уровнем холестирина.
    - **Кластер 1** состоит из людей с средним и  высоким уровнем холестирина.
    - **Кластер 2** состоит из людей с средним уровнем холестирина.
10. **CHOL_62** 
    - **Кластер 0** состоит из людей с средним и большим весом.
    - **Кластер 1** состоит из людей с средним весом.
    - **Кластер 2** состоит из людей с низким и средним весом. 
 
**Коэффициент силуэта для DBSCAN:** 0.4790

**Общее заключение:**

Алгоритм DBSCAN разделил пациентов на 3 кластера, при этом отбросив большую часть данных. Хотя Коэффициент силуэта неплохой, класстеризацию нельзя считать качественной.

### Метрики качества кластеризации

**Дополнительные метрики для оценки качества кластеризации**

Кроме коэффициента силуэта, существуют и другие метрики, которые помогают оценить качество кластеризации. Вот некоторые из них:

1. **Индекс Калински-Харабаза (Calinski-Harabasz Index):**
   - **Описание:** Измеряет отношение между разбросом между кластерами и разбросом внутри кластеров.
   - **Диапазон значений:** От 0 до бесконечности.
   - **Интерпретация:** Чем ближе к бесконечности больше, тем лучше раздленеие между кластерами

2. **Индекс Дэвиса-Болдина (Davies-Bouldin Index):**
   - **Описание:** Оценивает среднее "сходство" между каждым кластером и наиболее похожим на него другим кластером. Сходство основано на расстоянии между кластерами и их размером.
   - **Диапазон значений:** От 0 до бесконечности.
   - **Интерпретация:** Чем ближе к 0, тем лучшее разделение между кластерами.

3. **Коэффициент силуэта (Silhouette Score):**

    - **Описание:** Измеряет, насколько объект похож на свой кластер по сравнению с другими кластерами.
    - **Диапазон значений:** От -1 до 1.
    - **Интерпретация:** Значения, близкие к 1, указывают на хорошую кластеризацию. Значения около 0 означают, что кластеры перекрываются. Отрицательные значения указывают на возможные ошибки в кластеризации.

In [2]:
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score
# Кластеризация K-means
k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
labels_kmeans = kmeans.fit_predict(df)

# Индекс Калински-Харабаза
ch_score_kmeans = calinski_harabasz_score(df, labels_kmeans)
print(f"Индекс Калински-Харабаза для K-means: {ch_score_kmeans:.4f}")

# Индекс Дэвиса-Болдина
db_score_kmeans = davies_bouldin_score(df, labels_kmeans)
print(f"Индекс Дэвиса-Болдина для K-means: {db_score_kmeans:.4f}")

# Коэффициент силуэта
sh_score_kmeans = silhouette_score(df, labels_kmeans)
print(f"Коэффициент силуэта для K-means: {sh_score_kmeans:.4f}")

NameError: name 'KMeans' is not defined

In [3]:
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score

eps = 1
min_samples = 5

dbscan = DBSCAN(eps=eps, min_samples=min_samples)
labels_dbscan = dbscan.fit_predict(df)

# Исключаем шумовые точки (метки -1)
mask = labels_dbscan != -1
df_scaled_dbscan = df[mask]
labels_dbscan_filtered = labels_dbscan[mask]

# Проверяем, есть ли достаточное количество кластеров для расчета метрик
if len(set(labels_dbscan_filtered)) > 1:
    # Индекс Калински-Харабаза
    ch_score_dbscan = calinski_harabasz_score(df_scaled_dbscan, labels_dbscan_filtered)
    print(f"Индекс Калински-Харабаза для DBSCAN: {ch_score_dbscan:.4f}")

    # Индекс Дэвиса-Болдина
    db_score_dbscan = davies_bouldin_score(df_scaled_dbscan, labels_dbscan_filtered)
    print(f"Индекс Дэвиса-Болдина для DBSCAN: {db_score_dbscan:.4f}")

    # Коэффициент силуэта
    sh_score_dbscan = silhouette_score(df_scaled_dbscan, labels_dbscan_filtered)
    print(f"Коэффициент силуэта для DBSCAN: {sh_score_dbscan:.4f}")
else:
    print("Недостаточно кластеров для вычисления метрик для DBSCAN.")


NameError: name 'df' is not defined

#### Визуальный пример

In [4]:
# Диапазон значений k
k_range = range(2, 13)

# Списки для хранения метрик
ch_scores = []
db_scores = []
silhouette_scores = []

for k in k_range:
    # Создаем и обучаем модель KMeans
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(df)

    # Вычисляем метрики
    ch_score = calinski_harabasz_score(df, labels)
    db_score = davies_bouldin_score(df, labels)
    silhouette_avg = silhouette_score(df, labels)

    # Сохраняем метрики
    ch_scores.append(ch_score)
    db_scores.append(db_score)
    silhouette_scores.append(silhouette_avg)

# Построение графиков
plt.figure(figsize=(18, 5))

# График Индекса Калински-Харабаза
plt.subplot(1, 3, 1)
plt.plot(k_range, ch_scores, marker='o')
plt.title('Индекс Калински-Харабаза')
plt.xlabel('Количество кластеров k')
plt.ylabel('Значение метрики')

# График Индекса Дэвиса-Болдина
plt.subplot(1, 3, 2)
plt.plot(k_range, db_scores, marker='o')
plt.title('Индекс Дэвиса-Болдина')
plt.xlabel('Количество кластеров k')
plt.ylabel('Значение метрики')

# График Коэффициента силуэта
plt.subplot(1, 3, 3)
plt.plot(k_range, silhouette_scores, marker='o')
plt.title('Коэффициент силуэта')
plt.xlabel('Количество кластеров k')
plt.ylabel('Значение метрики')

plt.tight_layout()
plt.show()


NameError: name 'KMeans' is not defined

# Пример обработки текстовых данных

In [5]:
pip install nltk pymorphy3 scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import pymorphy3

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


#### Инициализация текстовых анализаторов

In [80]:
# Инициализируем лемматизатор и список стоп-слов
morph = pymorphy3.MorphAnalyzer()
russian_stopwords = stopwords.words('russian')

# Чтение данных из файла
with open('articles.txt', 'r', encoding='utf-8') as f:
    content = f.read()

# Разделяем данные на статьи
articles_raw = content.split('=====')

# Извлекаем заголовки и тексты статей
titles = []
texts = []

for article in articles_raw:
    lines = article.strip().split('\n')
    if len(lines) >= 6:
        # Извлекаем заголовок и текст статьи
        title = lines[4]
        text = ' '.join(lines[5:])
        titles.append(title)
        texts.append(text)

In [81]:
texts[:2]

['Большинство из нас видит Отечественную войну 1812 года глазами ее величайшего популяризатора – Льва Толстого. Формально «Война и мир» – книга художественная, но и автор, и многие читатели воспринимали ее как эпическое полотно из реального мира, в которое Толстой просто вплел судьбы кое-каких персонажей помельче. Из-за «толстовизации» истории Отечественной войны многие до сих пор считают, что Кутузов как полководец действовал мудро. Якобы он не хотел давать Наполеону Бородинского сражения, планируя скорее отдать Москву, и только под напором Александра I и двора это сражение он дал. Более того, Кутузов не хотел жертв со стороны русской армии и поэтому избегал решительных сражений с французами при их отступлении вдоль Старой смоленской дороги, и также поэтому не окружил их под Красным, еще в глубине России, где до границы было очень далеко. По той же причине же он не хотел и решительного сражения с Наполеоном на Березине, не гнал вперед свои уставшие войска, и от этого разгром Бонапарта

#### Предварительная обработка текстовых данных

1. преобразуем в lowercase
2. оставляем только слова
3. токенизируем текста
4. преобразуем слова в начальную форму

In [82]:
russian_stopwords

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [83]:
def preprocess_text(text):
    # Приводим к нижнему регистру
    text = text.lower()
    # Удаляем пунктуацию и цифры
    text = re.sub(r'[^а-яё\s]', '', text)
    # Токенизация
    words = text.split()
    # Удаление стоп-слов и лемматизация
    lemmatized = []
    for word in words:
        if word not in russian_stopwords:
            lemma = morph.parse(word)[0].normal_form
            lemmatized.append(lemma)
    return ' '.join(lemmatized)

# Применяем предобработку к каждому тексту
texts_preprocessed = [preprocess_text(text) for text in texts]


In [84]:
texts_preprocessed[:2]

['большинство видеть отечественный война год глаз великий популяризатор лев толстой формально война мир книга художественный автор многие читатель воспринимать эпический полотно реальный мир который толстой просто вплести судьба коекакий персонаж мелкий изз толстовизация история отечественный война многие сей пора считать кутузов полководец действовать мудро якобы хотеть давать наполеон бородинский сражение планировать скорее отдать москва напор александр двор это сражение дать кутузов хотеть жертва сторона русский армия поэтому избегать решительный сражение француз отступление вдоль старый смоленский дорога также поэтому окружить красный глубина россия граница очень далеко тот причина хотеть решительный сражение наполеон березина гнать вперёд свой устать войско разгром бонапарт россия полный сопровождаться пленение осень год сожаление весь вышеперечисленный лев толстой сыграть плохой услуга популяризация российский история сегодня достоверно известно кутузов планировать дать решительн

#### Преобразуем в векторный вид

In [110]:

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(dtype = np.float32) #!!!
X = vectorizer.fit_transform(texts_preprocessed)


In [103]:
# Параметры DBSCAN могут требовать подбора
dbscan = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
labels = dbscan.fit_predict(X)


In [104]:
# Создаем DataFrame с заголовками и метками кластеров
df = pd.DataFrame({'title': titles, 'label': labels})

# Выводим количество кластеров
unique_labels = set(labels)
n_clusters = len(unique_labels) - (1 if -1 in labels else 0)
print(f'Количество кластеров: {n_clusters}')

# Анализируем кластеры
for label in unique_labels:
    if label == -1:
        continue  # Пропускаем шум
    cluster_titles = df[df['label'] == label]['title'].head(3)
    print(f'\nКластер {label}:')
    for title in cluster_titles:
        print(f'- {title}')


Количество кластеров: 746

Кластер 0:
- AstraZeneca и разработчики российского «Спутника-V» испытают комбинированный вариант своих вакцин от коронавируса (Upd.) 
- Обнаружен сверхзаразный штамм китайского птичьего гриппа H7N9 
- Птичий грипп, год 1918-й 

Кластер 1:
- В США построят телескоп для поиска «второй Земли» 
- Ученые заметили циклическую активность у Проксимы Центавра 
- Рядом с ближайшей к нам звездой могут существовать землеподобные планеты 

Кластер 2:
- Вакцину от коронавируса AstraZeneca переименовали во избежание негативных ассоциаций 
- Европейский регулятор подтвердил безопасность вакцины от AstraZeneca. Но у некоторых стран остались вопросы 
- Запад массово приостанавливает вакцинацию от коронавируса препаратом AstraZeneca 

Кластер 3:
- Впервые в мире термоядерную плазму протестировали в токамаке нового поколения 
- Создан самый большой в мире термоядерный стелларатор 
- Американцы совершили революцию в области термоядерного синтеза, – СМИ 

Кластер 4:
- Физики МФТИ


Кластер 612:
- Рыбы впервые прошли зеркальный тест 
- Рыбы прошли «зеркальный тест», но ученые все равно сомневаются, есть ли у них самосознание 

Кластер 613:
- NASA обозначило последние сроки поисковой миссии Opportunity 
- Марсоход Opportunity заметили с орбиты Марса 

Кластер 614:
- Зонд New Horizons впервые увидел свою последнюю цель 
- Зонд New Horizons исследовал объект из пояса Койпера. Что дальше? 
- Происхождение астероида Ультима Туле озадачило исследователей 

Кластер 615:
- Ученые: алкоголь вреден в любых дозах 
- ВОЗ: алкоголь — причина каждой 20-й смерти в мире 

Кластер 616:
- Сверхпроводимость против ферромагнетизма: сыграли вничью 
- Сверхпроводящий ферромагнетик вскрывает карты 

Кластер 617:
- Ученые назвали крупнейшую птицу в истории 
- Величайшие птицы в истории оказались практически слепы 

Кластер 618:
- Стресс ухудшил память и уменьшил размер мозга у людей зрелого возраста 
- Ученые показали разницу между утренним и вечерним стрессом 
- Исследование показало, 

In [89]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [105]:
X

<22954x96540 sparse matrix of type '<class 'numpy.float32'>'
	with 3513841 stored elements in Compressed Sparse Row format>

In [107]:
from sklearn.decomposition import PCA
import plotly.graph_objects as go

# Применяем PCA для уменьшения размерности до 3 компонент
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X.toarray())

# Создаем DataFrame для визуализации
df_plot = pd.DataFrame({
    'PC1': X_pca[:, 0],
    'PC2': X_pca[:, 1],
    'PC3': X_pca[:, 2],
    'title': titles,
    'label': labels
})

# Создаем 3D Scatter Plot
fig = go.Figure(data=[go.Scatter3d(
    x=df_plot['PC1'],
    y=df_plot['PC2'],
    z=df_plot['PC3'],
    mode='markers',
    marker=dict(
        size=5,
        color=df_plot['label'],
        colorscale='Viridis',
        opacity=0.8
    ),
    text=df_plot['title'],
    hovertemplate='%{text}<extra></extra>'
)])

# Настраиваем внешний вид графика
fig.update_layout(
    title='3D PCA визуализация статей',
    scene=dict(
        xaxis_title='PC1',
        yaxis_title='PC2',
        zaxis_title='PC3'
    ),
    margin=dict(l=0, r=0, b=0, t=50)
)

# Генерируем кадры для анимации вращения графика
frames = []
for angle in range(0, 360, 10):
    camera = dict(
        eye=dict(x=2*np.cos(np.radians(angle)), y=2*np.sin(np.radians(angle)), z=1.5)
    )
    frames.append(go.Frame(layout=dict(scene_camera=camera)))

# Добавляем кадры в график
fig.frames = frames

# Добавляем кнопки управления анимацией
fig.update_layout(
    updatemenus=[dict(
        type='buttons',
        showactive=False,
        y=0.8,
        x=1.05,
        xanchor='left',
        yanchor='top',
        buttons=[dict(label='Вращать',
                      method='animate',
                      args=[None, dict(frame=dict(duration=50, redraw=True),
                                       transition=dict(duration=0),
                                       fromcurrent=True,
                                       mode='immediate')])]
    )]
)

# Отображаем график
fig.show()


MemoryError: Unable to allocate 8.26 GiB for an array with shape (22954, 96540) and data type float32

Памяти не хватает.